In [6]:
import psycopg2
import pandas as pd
from datetime import timedelta
from dateutil import parser


In [7]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.util import deprecations
from sqlalchemy.sql import text
deprecations.SILENCE_UBER_WARNING = True
# create database connection
import psycopg2
import psycopg2.extras
from typing import Iterator, Dict, Any

def insert_execute_batch(connection, ssql, df_dict) -> None:
    
    connection = psycopg2.connect(
        host="98.98.117.105",
        port='5432',
        database="medols",
        user="postgres",
        password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    )

    try:
        connection.autocommit = True
        with connection.cursor() as cursor:
            psycopg2.extras.execute_batch(cursor, ssql, df_dict)

    except:
        # Rollback changes if there's an error
        print(f"Error executing query: {e}")

    finally:
        connection.close()

def execute_query_psql(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [8]:
df = execute_query_psql("select id, publish_date from news_tbn_medol where converted_date is null")

In [9]:
from datetime import timedelta
from dateutil import parser
import pandas as pd

def convert_dates_final_adjustment(date_str):
    if pd.isna(date_str) or not isinstance(date_str, str):
        return None
    original_date_str = date_str  # Store the original date string
    date_str = date_str.replace("WIB", "").replace("WITA", "").replace('-', '').strip()
    date_str = ' '.join(date_str.split())  # Normalize whitespace
    try:
        date_time = parser.parse(date_str, fuzzy=True)
        if "WITA" in date_str:
            date_time += timedelta(hours=1)  # Adjust for WITA timezone
        return date_time
    except Exception as e:
        print(f"Error parsing date '{original_date_str}': {e}")
        return None

# Example of how this function might be called with a problematic date string:
# convert_dates_final_adjustment("32 February 2023 - 25:61 WIB")


In [18]:
x = convert_dates_final_adjustment('Senin, 26 Februari 2024 16:05 WIB')

Error parsing date 'Senin, 26 Februari 2024 16:05 WIB': bad month number 26; must be 1-12: Senin, 26 Februari 2024 16:05


In [16]:
print(x)

None


In [5]:
def update_database(row):
    update_query = """
    UPDATE news_tbn_medol
    SET converted_date = '%s'
    WHERE id = %s
    """
    update_query = update_query %(row['converted_date'], row['id'])
    # print(update_query)
    execute_query_psql(update_query)
    
if len(df) > 0:
    # Apply the conversion function to the 'publish_date' column
    df['converted_date'] = df['publish_date'].apply(convert_dates_final_adjustment)
    df.apply(update_database, axis=1)
    # print(df)